# The analyzer of the possibility of the reversibility of words' mutation 

In [1]:
url_dataset = 'https://github.com/brannondorsey/naive-hashcat/releases/do|wnload/data/rockyou.txt'

We have to install wget package in order to download the dataset. Then import codecs module in order to read file with a necessary encoding, ANSI in this case.

In [2]:
pip install wget

Note: you may need to restart the kernel to use updated packages.


In [3]:
import wget
import os
file = 'rockyou.txt'
if not os.path.isfile(file):
    file = wget.download(url_dataset)

In [4]:
import codecs
with codecs.open(file, 'r', 'ANSI') as f:
    data = f.read().splitlines()

The dataset consists of more 14 millions words, so we have to get a trim of dataset to ease calculating and waste less time. Then clear the dataset of empty passwords.

In [5]:
passwords = [passwd for passwd in data[::4000] if passwd]
len(passwords)

3587

# Split and analyze lexemes

Здесь имеются около 125к слов, расположенных в порядке убывания частоты встречаемости. На основе этого словаря высчитываются wordcost по закону Ципфа.

In [6]:
url = 'https://github.com/keredson/wordninja/blob/master/wordninja/wordninja_words.txt.gz'

In [7]:
with open('wordninja_words.txt', 'r') as f:
    frequency_words = f.read().splitlines()

In [8]:
from math import log

# Zipf's law
words = frequency_words
wordcost = dict((k, log((i+1)*log(len(words)))) for i,k in enumerate(words))

Первые 10 наиболее встречающихся слов.

In [9]:
frequency_words[:10]

['the', 'of', 'in', 'a', 'and', 'is', 'to', 'was', 'it', 'for']

В этом пакете реализована функция разделения слова на лексемы, основываясь на том же словаре

In [10]:
pip install wordninja

Note: you may need to restart the kernel to use updated packages.


In [11]:
import wordninja

## Examples

In [12]:
wordninja.split('thisismypassword')

['this', 'is', 'my', 'password']

Функция выводит суммарную стоимость раздеделения на лексемы. Она поможет определить наиболее вероятный вариант разделения, уменьшая общую стоимость. Ниже приведен пример уменьшения стоимости при исправлении некорректных символов.

In [14]:
def get_total_cost(string):
    return sum([wordcost.get(word, 100) for word in wordninja.split(string)])

In [15]:
print(wordninja.split('ths11smypa$$w0rd3'), get_total_cost('ths11smypa$$w0rd3'))
print(wordninja.split('thsi1smypa$$w0rd3'), get_total_cost('thsi1smypa$$w0rd3'))
print(wordninja.split('thsi1smypas$w0rd3'), get_total_cost('thsi1smypas$w0rd3'))
print(wordninja.split('thsismypassw0rd3'), get_total_cost('thsismypassw0rd3'))
print(wordninja.split('thsismypassword3'), get_total_cost('thsismypassword3'))
print(wordninja.split('thisismypassword3'), get_total_cost('thisismypassword3'))
print(wordninja.split('thisismypassworde'), get_total_cost('thisismypassworde'))
print(wordninja.split('thsismypassworde'), get_total_cost('thsismypassworde'))
print(wordninja.split('thisismypassword'), get_total_cost('thisismypassword'))

['th', 's', '11', 's', 'my', 'pa', 'w', '0', 'rd', '3'] 185.4656164900615
['th', 'si', '1', 's', 'my', 'pa', 'w', '0', 'rd', '3'] 105.2123667183307
['th', 'si', '1', 's', 'my', 'pas', 'w', '0', 'rd', '3'] 102.55225597951761
['th', 's', 'is', 'my', 'pass', 'w', '0', 'rd', '3'] 82.5043512191454
['th', 's', 'is', 'my', 'password', '3'] 51.24674348376489
['this', 'is', 'my', 'password', '3'] 44.303267203244076
['this', 'is', 'my', 'password', 'e'] 37.96537057462295
['th', 's', 'is', 'my', 'password', 'e'] 44.90884685514377
['this', 'is', 'my', 'password'] 30.094761311203957


Основная идея нахождения лексем состоит в следующем:  
1) находим все подменяемые символы ($, 1, @ и т.п.) и выделяем их индексы  
2) создаем всевозможные сочетания индексов, чтобы найти оптимальный вариант  
3) преобразовываем найденные варианты и высчитываем их сумму  
4) находим минимальную сумму и выводим лучший результат

Следующий пример показывает пример нахождения всех символов.

In [16]:
original = 'Ths_1s_mypa$$w0rd3'

In [18]:
get_total_cost(original)

192.51932115113203

In [20]:
import re
re.findall(r'[\W\d\s]', original)

['1', '$', '$', '0', '3']

## Analyze lexemes

Данная функция подменяет все найденные по переданным индексам символы в comparison. Остальные найденные некорректные символы удаляются из строки.

In [21]:
def leet_transform(string, indices):
    comparison = {
        '$': 's',
        '1': 'i',
        '0': 'o',
        '3': 'e',
        '@': 'a'
    }
    
    for index in indices:
        symbol = comparison.get(string[index])
        
        if not symbol:
            continue
            
        string = string[:index] + symbol + string[index+1:]
          
    return re.sub(r'[\W\d\s_]+', r'', string)

In [22]:
from itertools import combinations

Функция возвращает все индексы некорректных символов в строке

In [23]:
def get_indices_incorrect_symbols(word):
    arr = []
    index = 0
    for char in re.findall(r'[\W\d\s]', word):
        index = word.index(char, index)
        arr.append(index)
        index += 1
    
    return arr

Функция возвращает всевозможные сочетания индексов

In [24]:
def get_all_combinations(indices):
    coms = []
    for i in range(1, len(indices)+1):
        coms.extend(combinations(indices, i))
        
    return coms

Данная функция высчитывает сумму для всех найденных комбинаций и возвращает получившуюся строку с наименьшей суммой 

In [28]:
def get_correct_password(word):
    indices = get_indices_incorrect_symbols(word)
    
    corrected_password = [1000, '']
    
    all_combinations = get_all_combinations(indices)
    # Defines an empty combination
    all_combinations.append([])
    
    for combinations in all_combinations:
        prepared_str = leet_transform(word, combinations)

        total_cost = get_total_cost(prepared_str.lower())

        if total_cost < corrected_password[0]:
            corrected_password = [total_cost, prepared_str]

    return corrected_password[1]

Пример применения данной функции на наборе из 3500 паролей

In [29]:
correct_passwords_list = [(password, get_correct_password(password)) for password in passwords]

In [30]:
correct_passwords_list[::10]

[('123456', ''),
 ('mocosa', 'mocosa'),
 ('271005', ''),
 ('28defebrero', 'defebrero'),
 ('frumushika', 'frumushika'),
 ('judy22', 'judy'),
 ('locoboy', 'locoboy'),
 ('allyson2', 'allyson'),
 ('Daniel7', 'Daniel'),
 ('copper84', 'copper'),
 ('my babe', 'mybabe'),
 ('Buffett', 'Buffett'),
 ('nuber1', 'nuber'),
 ('dababy1', 'dababy'),
 ('081873', ''),
 ('plunket', 'plunket'),
 ('jonny!', 'jonny'),
 ('charmoso', 'charmoso'),
 ('511522', ''),
 ('verizon25', 'verizon'),
 ('saullopez', 'saullopez'),
 ('nikolas95', 'nikolas'),
 ('littley', 'littley'),
 ('iusti', 'iusti'),
 ('elmejor20', 'elmejor'),
 ('budakdak', 'budakdak'),
 ('TRAVIS11', 'TRAVIS'),
 ('5151984', ''),
 ('11714', ''),
 ('wuhtever89', 'wuhtever'),
 ('tstj5014', 'tstj'),
 ('takki329', 'takki'),
 ('skejterka', 'skejterka'),
 ('salmon4', 'salmon'),
 ('random360', 'random'),
 ('percylee', 'percylee'),
 ('ninan', 'ninan'),
 ('monajojie', 'monajojie'),
 ('maryanne14', 'maryanne'),
 ('louryn', 'louryn'),
 ('labrat77', 'labrat'),
 ('kan

Следующий пример демонстрирует работу функции на возможном пароле "the15ths1s_my_p@$$w0rd33". Базовым словом было "thethisismypassword". Для исправления найденного слова данной функцией "thethsismypassword" подразумевается использовать BK-Tree, использующая в виде меры расстояние Дамерау-Левенштейна.

Для данного примера необходимо исправить слова 'th' и 's', получив 'this'. Идея исправления состоит в следующем:  
1) с первого слова по последнее проходить по порядку, склеивая их  
2) получившиеся слова пытаться исправить построенным BK-деревом, находя наименьшее по стоимости слово  
3) определить оптимальный вариант и исправить ошибку, получив вероятное решение  

In [41]:
corrected = get_correct_password("the15ths1s_my_p@$$w0rd33")
print(wordninja.split(corrected))

['the', 'th', 's', 'is', 'my', 'password']


In [35]:
import pybktree as bk
from similarity.damerau import Damerau
import pickle
import os

filename = 'tree.pickle'

if os.path.isfile(filename):
    with open(filename, 'rb') as file:
        tree = pickle.load(file)
else:
    tree = bk.BKTree(Damerau().distance, frequency_words)
    with open(filename, 'wb') as file:
        pickle.dump(tree, file)

In [36]:
finded_words = tree.find('ths', 1)
finded_words[:5]

[(0.0, 'ths'), (1, 'the'), (1, 'this'), (1, 'th'), (1, 'thus')]

In [38]:
arr = [(get_total_cost(it[1]), it[1]) for it in finded_words]
sorted(arr)[:5]

[(2.4634374919588784, 'the'),
 (5.507959929682301, 'this'),
 (7.154785374188022, 'th'),
 (7.760088327973973, 'thes'),
 (9.562639235511972, 'thus')]